In [68]:
import numpy as np

class Layer:
    def __init__(self, input_left_mt: np.ndarray, input_right_mt: np.ndarray, var_index):
        self.input_left_mt = input_left_mt
        self.input_right_mt = input_right_mt
        self.var_index = var_index
        self.downer_block, self.upper_block, self.zero_ineq = self.distinguishing()

    @property
    def total_right(self):
        right_mt = self.input_right_mt - self.input_left_mt
        right_mt[:, self.var_index] = 0
        return right_mt
    
    @property
    def total_left(self):
        left_mt = self.input_left_mt - self.input_right_mt
        left_mt[:,[i for i in range(left_mt.shape[1]) if i != self.var_index]] = 0
        return left_mt

    def distinguishing(self):
        possitive_ineq_index = self.total_left[:,self.var_index] > 0
        negative_ineq_index = self.total_left[:,self.var_index] < 0
        zero_ineq_index = self.total_left[:,self.var_index] == 0

        upper_block = self.total_right[possitive_ineq_index] / self.total_left[possitive_ineq_index, self.var_index][:, None]
        downer_block = self.total_right[negative_ineq_index] / self.total_left[negative_ineq_index, self.var_index][:, None]

        return downer_block, upper_block, -self.total_right[zero_ineq_index]

class Scheme:
    def __init__(self, input_matrix: np.ndarray):
        self.input_matrix = input_matrix
        self.nvar = input_matrix.shape[1]
        self.nequal = input_matrix.shape[0]

input_matrix = np.array([
    [1,2,3],        #   a + 2b + 3c >= 0 
    [-2,0,1],       # -2a + 0b +  c >= 0
    [2,1,0]         #  2a +  b + 0c >= 0
])

# input_matrix[:,1]
lay = Layer(input_left_mt=input_matrix, input_right_mt=np.zeros([3,3]), var_index=0)
print(lay.upper_block)
print(lay.downer_block)
print(lay.zero_ineq)

[[ 0.  -2.  -3. ]
 [ 0.  -0.5  0. ]]
[[-0.  -0.   0.5]]
[]
